In [8]:
import cv2 as cv
import numpy as np

In [9]:
def empty_callback(value):
    pass

In [10]:
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

In [11]:
capture1 = False
capture2 = False

In [12]:
img = np.zeros((512, 512, 1), dtype=np.uint8)
background = np.zeros((512, 512, 1), dtype=np.uint8)
foreground = np.zeros((512, 512, 1), dtype=np.uint8)
diff = np.zeros((512, 512, 1), dtype=np.uint8)
cv.namedWindow('Background Image')
cv.namedWindow('Current Image')
cv.namedWindow('Foreground Image')
cv.createTrackbar('x', 'Foreground Image', 0, 255, empty_callback)

In [ ]:
#Task Absdiff type a to save background, type x to save current image. 
#Each time you type x you see difference between current and last frame.
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, dsize=(512, 512))
    # Display the resulting frame
    cv.imshow('Background Image', background)
    cv.imshow('Current Image', img)
    cv.imshow('Foreground Image', foreground)
    key_code = cv.waitKey(10)
    if key_code == 27:
        # escape key pressed
        break
    if key_code == ord('x'):
        print('Save current image')
        current = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        current = cv.resize(current, dsize=(512, 512))
        capture1 = True
    if key_code == ord('a'):
        print('Save background')
        background = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        background = cv.resize(background, dsize=(512, 512))
        capture2 = True
    if key_code == ord('r'):
        print('Reset')
        background = np.zeros((512, 512, 1), dtype=np.uint8)
        foreground = np.zeros((512, 512, 1), dtype=np.uint8)
        capture1 = False
        capture2 = False
    if capture1 and capture2:
        diff = cv.absdiff(current, background)
        capture1 = False
    # get current positions of four trackbars
    th = cv.getTrackbarPos('x', 'Foreground Image')
    ret,foreground = cv.threshold(diff,int(th),255,cv.THRESH_BINARY)

In [ ]:
#Task Absdiff type a to save background, then is running in continous mode
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, dsize=(512, 512))
    # Display the resulting frame
    cv.imshow('Background Image', diff)
    cv.imshow('Current Image', img)
    cv.imshow('Foreground Image', foreground)
    key_code = cv.waitKey(10)
    if key_code == 27:
        # escape key pressed
        break
    if not capture1:
        print('Save background')
        background = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        background = cv.resize(background, dsize=(512, 512))
        capture1 = True
    if key_code == ord('r'):
        print('Reset')
        background = np.zeros((512, 512, 1), dtype=np.uint8)
        foreground = np.zeros((512, 512, 1), dtype=np.uint8)
        capture1 = False
    if capture1:
        current = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        current = cv.resize(current, dsize=(512, 512))
        diff = cv.absdiff(current, background)
    # get current positions of four trackbars
    th = cv.getTrackbarPos('x', 'Foreground Image')
    ret,foreground = cv.threshold(diff,int(th),255,cv.THRESH_BINARY)

In [ ]:
def median_model(img, background):
    new_back = background.copy()
    new_back[background<img] += 1
    new_back[background>img] -= 1
    new_back[new_back>255] = 255
    new_back[new_back<0] = 0
    return new_back

In [ ]:
# Approximate median approach in continous mode
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, dsize=(512, 512))
    # Display the resulting frame
    cv.imshow('Background Image', background)
    cv.imshow('Current Image', img)
    cv.imshow('Foreground Image', foreground)
    key_code = cv.waitKey(10)
    if key_code == 27:
        # escape key pressed
        break
    if not capture1:
        print('Capture background and current frame')
        background = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        background = cv.resize(background, dsize=(512, 512))
        current = cv.resize(background, dsize=(512, 512))
        background = median_model(current, background)
        capture1 = True
    if key_code == ord('r'):
        print('Reset')
        background = np.zeros((512, 512, 1), dtype=np.uint8)
        foreground = np.zeros((512, 512, 1), dtype=np.uint8)
        capture1 = False
    if capture1:
        current = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        current = cv.resize(current, dsize=(512, 512))
        background = median_model(current, background)
        diff = cv.absdiff(current, background)
    # get current positions of four trackbars
    th = cv.getTrackbarPos('x', 'Foreground Image')
    ret,foreground = cv.threshold(diff,int(th),255,cv.THRESH_BINARY)

In [ ]:
cv.destroyAllWindows()

In [13]:
arg = 'MOG2'
if arg == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, dsize=(512, 512))
    # Display the resulting frame
    fgMask = backSub.apply(frame)
    cv.imshow('Foreground Image', fgMask)
    cv.imshow('Background Image', background)
    cv.imshow('Current Image', img)
    key_code = cv.waitKey(10)
    if key_code == 27:
        # escape key pressed
        break
#     th = cv.getTrackbarPos('x', 'Foreground Image')
#     ret, foreground = cv.threshold(diff,int(th),255,cv.THRESH_BINARY)